- [Linear Regression](#linear-regression)
- [Logistic Regression](#logistic-regression)

In [2]:
import probpy as pp
import numpy as np
import numba
import random

linear regression
---

In [2]:
%%time
def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]

w = [5, -2, 1, 5] # True underlying model
x = np.random.rand(100, 3) * 10
y = predict(w, x) + pp.normal.sample(mu=0, sigma=1, size=100).reshape(-1)



prior = pp.multivariate_normal.med(mu=np.ones(4) * 0, sigma=np.eye(4) * 5)
likelihood = pp.unilinear.med(sigma=1) # There exist an implementation for linear because it has a conjugate prior

for i in range(100):
    data = (y[i], x[i])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, priors=prior)
    
    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - predict(w_approx, x)).mean(), 
              "True MSE", np.square(y - predict(w, x)).mean())
        print()

Parameter Estimate [3.56436856 0.92738658 2.68264725 0.42010986]
Prior MSE 224.32842387613914 True MSE 0.8524988178683403

Parameter Estimate [ 5.20447922 -1.7797635   1.1071891   3.17947455]
Prior MSE 2.2127831858489238 True MSE 0.8524988178683403

Parameter Estimate [ 4.95543377 -2.08204834  1.06431287  5.24076141]
Prior MSE 0.8659901804201242 True MSE 0.8524988178683403

Parameter Estimate [ 4.94401297 -2.09351187  1.06453333  5.42301255]
Prior MSE 0.8739487224827588 True MSE 0.8524988178683403

Parameter Estimate [ 4.93212834 -2.08456259  1.0682134   5.46967396]
Prior MSE 0.8751294691695394 True MSE 0.8524988178683403

Parameter Estimate [ 4.94339976 -2.04358628  1.07064414  5.16403092]
Prior MSE 0.8452392867069826 True MSE 0.8524988178683403

Parameter Estimate [ 4.94634446 -2.04771106  1.04337223  5.29774805]
Prior MSE 0.8399504179691826 True MSE 0.8524988178683403

Parameter Estimate [ 4.97269293 -2.03357536  1.03182458  5.21298402]
Prior MSE 0.8328712828498225 True MSE 0.852498

logistic regression
---

In [3]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w): # This should be as fast as possible since it is the bottle-neck
    # It is possible to rewrite this loop in pure numpy with some broadcasting magic but 
    # I think this example is more instructive
        
    result = []
    for _w in w:
        prediction = sigmoid(predict(_w, x))
        result.append(pp.normal.p(y - prediction, mu=0, sigma=1.0))
        
    return result

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   match_moments_for=pp.multivariate_normal,
                                   batch=50,
                                   samples=5000,
                                   burn_in=500, 
                                   energies=1.0)

    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Parameter Estimate [-0.23601055  0.05588377  0.34628096 -0.71836587]
Prior MSE 0.0995996134116718 True MSE 0.031297186525624035

Parameter Estimate [-2.04545956  1.82695563  3.30731125 -1.82869796]
Prior MSE 0.032569237871609785 True MSE 0.031297186525624035

Parameter Estimate [-2.47703965  2.42116223  3.879914   -2.25933299]
Prior MSE 0.030886236193817557 True MSE 0.031297186525624035

Parameter Estimate [-2.76113245  2.88005064  4.22459557 -2.48620847]
Prior MSE 0.03170789237288279 True MSE 0.031297186525624035

Parameter Estimate [-3.35884969  3.13486393  4.94900395 -2.85277206]
Prior MSE 0.03205663697020225 True MSE 0.031297186525624035

CPU times: user 1min 33s, sys: 2min 24s, total: 3min 57s
Wall time: 60 s


Logistic Regression + Numpy magic ~ 10x speed-up
---

In [8]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   match_moments_for=pp.multivariate_normal,
                                   batch=50,
                                   samples=5000,
                                   mixing=500, 
                                   energies=1.0)

    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Parameter Estimate [-0.53601026  0.28077979  0.46164084 -0.8330969 ]
Prior MSE 0.0782992400216686 True MSE 0.027590952878289207

Parameter Estimate [-1.96945747  1.46968586  3.37183621 -1.93631875]
Prior MSE 0.02917472725253408 True MSE 0.027590952878289207

Parameter Estimate [-2.54491043  2.14892638  4.12655498 -2.41129613]
Prior MSE 0.026715544705368424 True MSE 0.027590952878289207

Parameter Estimate [-2.78140459  2.54598524  4.3280773  -2.57244346]
Prior MSE 0.0262718273798136 True MSE 0.027590952878289207

Parameter Estimate [-3.13342456  2.65961273  4.56699265 -2.7096384 ]
Prior MSE 0.026869735122740566 True MSE 0.027590952878289207

CPU times: user 14.8 s, sys: 22.8 s, total: 37.6 s
Wall time: 9.41 s


Logistic Regression without moment-matching
---

In [12]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=5,
                                   samples=1000,
                                   mixing=100, 
                                   energies=1.0)

    if i % 10 == 0:
        modes = pp.mode(prior) # modes are sorted in order first is largest

        print("Number of modes", len(modes))
        w_approx = modes[0]
        
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Number of modes 5
Parameter Estimate [ 0.25289025  1.07358778  0.22271974 -1.45592379]
Prior MSE 0.08989462571191517 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-1.6441759   1.24321202  2.4961304  -1.67858393]
Prior MSE 0.03433557116653359 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-3.08648632  2.71105106  2.76933978 -1.44667161]
Prior MSE 0.030504307227619015 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-3.40101005  2.76046226  3.64808157 -1.80800666]
Prior MSE 0.027947714687514585 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-3.56976171  2.86013503  3.1125747  -1.88175391]
Prior MSE 0.031942999881937405 True MSE 0.02557124439040531

CPU times: user 14 s, sys: 747 ms, total: 14.8 s
Wall time: 13.9 s


Logistic Regression with other posterior estimate
---

In [4]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(5):
    data = (y, x)
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=50,
                                   samples=20000,
                                   mixing=5000, 
                                   energies=1.0,
                                   classical_mcmc=False,
                                   normalize=False)

    modes = pp.mode(prior) # modes are sorted in order first is largest

    print("Number of modes", len(modes))
    for mode in modes:
        print(mode)

    w_approx = modes[0]

    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

Number of modes 4
[-1.63452554  0.96723127  3.27014492 -0.58126018]
[-3.04825833  0.57708452  0.73175645  0.1098704 ]
[-1.45124457  4.49943872  0.79745612 -1.81475637]
[-2.99148383  1.8024128   2.36494075 -1.97266855]
Parameter Estimate [-1.63452554  0.96723127  3.27014492 -0.58126018]
Prior MSE 0.0789040393623053 True MSE 0.023388310842695548

Number of modes 1
[-1.42618642  0.66182234  3.36814191 -1.29367691]
Parameter Estimate [-1.42618642  0.66182234  3.36814191 -1.29367691]
Prior MSE 0.04278299381733612 True MSE 0.023388310842695548

Number of modes 5
[-0.7544359   0.33087263  3.33454616 -1.34069678]
[-1.87471994  1.64245752  2.66915492 -1.9817099 ]
[-2.31721921  0.84380485  2.88495273 -1.14035784]
[-1.20948516  0.99006203  3.15938831 -1.5386404 ]
[-0.6351583   1.83321605  4.93629858 -3.3390042 ]
Parameter Estimate [-0.7544359   0.33087263  3.33454616 -1.34069678]
Prior MSE 0.05670034203796678 True MSE 0.023388310842695548

Number of modes 2
[-2.17359541  1.63739972  3.68686974 -1